In [1]:
import org.apache.spark.ml.classification.LogisticRegression

In [2]:
val trainPct = 0.8
val testPct = 1 - trainPct
val dataLoc = "/home/ubuntu/data/moons/"

trainPct = 0.8
testPct = 0.19999999999999996
dataLoc = /home/ubuntu/data/moons/


/home/ubuntu/data/moons/

In [3]:
val df = spark.read.option("header", "true").csv(dataLoc+"*.csv")
df.describe().show

+-------+--------------------+--------------------+--------------------+
|summary|               class|                   X|                   Y|
+-------+--------------------+--------------------+--------------------+
|  count|                5000|                5000|                5000|
|   mean|                 0.5|  0.4920739283996744| 0.24980917811368442|
| stddev|  0.5000500075012502|  0.9182629238525909|  0.5789451721138708|
|    min|0.000000000000000...|-1.00117901148862...|-1.00139457799930...|
|    max|1.000000000000000...|9.987326376696318...|9.983360574266741...|
+-------+--------------------+--------------------+--------------------+


df = [class: string, X: string ... 1 more field]


[class: string, X: string ... 1 more field]

In [4]:
// Cast the string fields to numerics
import org.apache.spark.sql.types.DecimalType
import org.apache.spark.sql.types.IntegerType

val precision = 10
val scale = 8
val df2 = df.withColumn("class", df("class").cast("float").cast(IntegerType))
.withColumn("x", df("X").cast(DecimalType(precision, scale)))
.withColumn("y", df("Y").cast(DecimalType(precision, scale)))

df2.describe().show

+-------+------------------+------------------+------------------+
|summary|             class|                 x|                 y|
+-------+------------------+------------------+------------------+
|  count|              5000|              5000|              5000|
|   mean|               0.5|    0.492073928418|    0.249809178078|
| stddev|0.5000500075012502|0.9182629238273076|0.5789451721684695|
|    min|                 0|       -1.96032021|       -1.31754580|
|    max|                 1|        2.87904526|        1.90263060|
+-------+------------------+------------------+------------------+



precision = 10
scale = 8
df2 = [class: int, x: decimal(10,8) ... 1 more field]


[class: int, x: decimal(10,8) ... 1 more field]

In [5]:
// Create a "features" vector for X and Y
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler()
  .setInputCols(Array("x", "y"))
  .setOutputCol("features")

val transformed = assembler.transform(df2)

transformed.show

+-----+-----------+-----------+--------------------+
|class|          x|          y|            features|
+-----+-----------+-----------+--------------------+
|    1| 1.08787248|-0.20521282|[1.08787248,-0.20...|
|    0|-0.79452014| 0.94437628|[-0.79452014,0.94...|
|    1| 0.25275002|-0.55257111|[0.25275002,-0.55...|
|    1| 1.88303838| 0.20460338|[1.88303838,0.204...|
|    0|-0.55615115| 1.38986852|[-0.55615115,1.38...|
|    1| 0.03783374| 0.59497768|[0.03783374,0.594...|
|    0|-0.92325788|-0.09656239|[-0.92325788,-0.0...|
|    0|-0.49051364| 1.36998264|[-0.49051364,1.36...|
|    1|-0.11272890|-0.49344063|[-0.1127289,-0.49...|
|    1| 2.47830680| 0.63353222|[2.4783068,0.6335...|
|    1| 1.98872986| 0.09737975|[1.98872986,0.097...|
|    0| 0.38236654| 0.84140897|[0.38236654,0.841...|
|    0|-0.97561951| 1.16236005|[-0.97561951,1.16...|
|    0| 0.10791343| 1.09045331|[0.10791343,1.090...|
|    1| 1.03539022|-0.00827754|[1.03539022,-0.00...|
|    1| 0.91384781|-0.64081650|[0.91384781,-0.

assembler = vecAssembler_217ad54248b2
transformed = [class: int, x: decimal(10,8) ... 2 more fields]


[class: int, x: decimal(10,8) ... 2 more fields]

In [11]:
// Create Train and Test sets
val Array(train, test) = transformed.randomSplit(Array(trainPct, testPct))

println("Train file has " + train.count + " records")
print("Test file has " + test.count + " records")

Train file has 4023 records
Test file has 977 records

train = [class: int, x: decimal(10,8) ... 2 more fields]
test = [class: int, x: decimal(10,8) ... 2 more fields]


[class: int, x: decimal(10,8) ... 2 more fields]

In [13]:
// Build and Fit the model against "train" data
val lr = new LogisticRegression()
  .setMaxIter(10)
  .setRegParam(0.3)
  .setElasticNetParam(0.8)
  .setLabelCol("class")
  .setFeaturesCol("features")

val lrModel = lr.fit(train)

println("X coefficient: "+ lrModel.coefficientMatrix.apply(0,0))
println("Y coefficient: "+ lrModel.coefficientMatrix.apply(0,1))

println("Intercept: " + lrModel.interceptVector(0))

X coefficient: 0.005169208020223935
Y coefficient: -0.49186555092064665
Intercept: 0.10815666795012131


lr = logreg_e0f52836e834
lrModel = logreg_e0f52836e834


logreg_e0f52836e834

In [41]:
val predictions = lrModel.transform(test)
val preds = predictions.select("x", "y","prediction")
preds.coalesce(1).write.option("header", "true").csv(dataLoc+"preds")
val actuals = test.select("x", "y", "class")
actuals.coalesce(1).write.option("header", "true").csv(dataLoc+"actuals")

predictions = [class: int, x: decimal(10,8) ... 5 more fields]
preds = [x: decimal(10,8), y: decimal(10,8) ... 1 more field]
actuals = [x: decimal(10,8), y: decimal(10,8) ... 1 more field]


[x: decimal(10,8), y: decimal(10,8) ... 1 more field]